# Homework 6
Calculating fairness statistics.

## Task 1

$Y$ - will use XAI\
$\hat{Y}$ - enrolled in training

### Demographic parity
$P(\hat{Y}|Blue) = 0.65$\
$P(\hat{Y}|Red) = 0.5$\
$\frac{0.65}{0.5} = 1.3$

### Equal opportunity
$P(\hat{Y} | Blue, Y) = 0.75$\
$P(\hat{Y} | Red, Y) = 0.5$\
$\frac{0.75}{0.5} = 1.5$

### Predictive rate parity
$P(Y | \hat{Y}, Blue) = \frac{60}{65}$\
$P(Y | \hat{Y}, Red) = 0.5$\
$\frac{\frac{60}{65}}{0.5} = 1.85$

## Task 2
For this homework, I use adult income dataset.

Main model used for evaluation is `XGBClassifier`. Experiments run also on `RandomForestClassifier`.

### Data description
+ age: continuous.
+ workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
+ fnlwgt: continuous.
+ education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
+ education-num: continuous.
+ marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
+ occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
+ relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
+ race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
+ sex: Female, Male.
+ capital-gain: continuous.
+ capital-loss: continuous.
+ hours-per-week: continuous.
+ native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
+ class: >50K, <=50K

### 2.1

Below, performance of `XGBClassifier` is presented on adult income dataset. Before running training, categorical features are one hot encoded.
Achieved results are later used as a baseline:
<br/><br/>

|  | recall | precision | f1 | accuracy | auc |
| :--- | :--- | :--- | :--- | :--- | :--- |
| XGBClassifier | 0.585543 | 0.790872 | 0.672893 | 0.863753 | 0.917741 |


### 2.2

I select gender as a variable that is later used for fairness analysis. `male` value is set as privileged. As it is seen below, model is not fair because 2 criterias exceed acceptable limits. Some of them are far from the set epsilon quite drastically.
<br/>

```
Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
            TPR       ACC       PPV       FPR       STP
female  0.89916  1.130435  1.049618  0.197183  0.313043
```

As it is seen on the plot, the model is far from being fair. The bias was detected among two metrics - `FPR` and `STP`.
Predictive equality ratio is equal to 0.197, which means that FPR is over 5 times greater for `females` than for `males` which is enormous difference.
Similarly, Statistical Parity Ratio is equal to 0.313 for `females` which makes it over 3 times more likely to predict that `females` generate income less than 50k than it predicts it for `males`.
<br/>

<img src="imgs/img2.2.png" alt="task1 solution" width="1300"/>

### 2.3
To verify if unfair results were the cause of model selection, we train another model but this time it is `RandomForestClassifier`.
In table below, we can see that accuracy is similar for `RandomForestClassifier` and `XGBClassifier`. However, `precision` achieved by `RandomForestClassifier` is significantly lower than the one obtained with `XGBC`. This means that fewer people that are predicted to have income <=50k actually have such income. This could imply that the `RandomForestClassifier` introduces much less fairness.

`Random Forest Classifier` scores:

|  | recall | precision | f1 | accuracy | auc |
| :--- | :--- | :--- | :--- | :--- | :--- |
| RandomForestClassifier | 0.642002 | 0.735064 | 0.685388 | 0.858942 | 0.90453 |


Fairness check indicates that all results are more or less similar, despite the model architecture choice. Predictive parity ratio is slightly closer to 1, Equal opportunity ratio is lower. The difference in fairness is insignificant, slightly on behalf of infairness.

<br/>
<img src="imgs/img2.3.png" alt="task1 solution" width="1300"/>


### 2.4
I verified two different bias mitigation techniques. Those are `resample` and `reweight`. The results are shown below.
Both techniques slightly improve model fairness, however both methods do not perform exactly the same. We can see that in case of detected bias in metrics, using `resample` method, there were 2 metrics found but using `reweight` method, there were 3 metrics found. Considering number of detected biases, `resample` method performs better in this exact situation.

We can notice, that metrics, which were close to 1, after applying any of techniques are more biased. However, what is important is that metrics which were far from being balanced, now are much closer to 1. Bias mitigation is not costless, however it improves fairness of achieved results significantly.

<br/>

Resample:

```
Bias detected in 2 metrics: PPV, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC      PPV   FPR       STP
female  1.245902  1.114734  0.77657  0.94  0.584158
```

Reweight:

```
Bias detected in 3 metrics: TPR, PPV, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV       FPR       STP
female  1.303309  1.114458  0.768772  1.088889  0.619289
```


<img src="imgs/img2.4.all.png" alt="task1 solution" width="1300"/>

### 2.5
Performance comparison:
<br/>

|                        | recall | precision | f1 | accuracy | auc |
|:-----------------------| :--- | :--- | :--- | :--- | :--- |
| XGBClassifier          | 0.585543 | 0.790872 | 0.672893 | 0.863753 | 0.917741 |
| RandomForestClassifier | 0.642002 | 0.735064 | 0.685388 | 0.858942 | 0.90453 |
| XGBClassifier resample | 0.570145 | 0.785967 | 0.660882 | 0.859965 | 0.911005 |
| XGBClassifier reweight | 0.56929 | 0.793206 | 0.662849 | 0.861398 | 0.911288 |

After testing four different models, among which two are classic tree models and two use bias mitigation techniques, we can see some interesting results. The most important thing to notice is that achieved `accuracy`, `f1 score` and `auc` are more or less the same. It implies that it would be beneficial to use model that is fair instead of any available model. We do not lose in performance but also gain in fairness. It is crucial to use bias mitigation techniques in socially subtle situations such as risk management or granting loans.


## Appendix

In [103]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import pandas as pd
import numpy as np

import dalex as dx

import plotly.express as px

from dalex.fairness import resample, reweight

In [104]:
df = pd.read_csv('adult.csv')
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [105]:
cat_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country']
num_cols = ['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss', 'hours-per-week']

df_tr = pd.get_dummies(df, columns=cat_cols, drop_first=True)
y = df_tr[['income']] == '>50K'
X = df_tr.drop('income', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42, shuffle=True, stratify=y)

In [106]:
model = XGBClassifier(
    n_estimators=50,
    max_depth=2,
    use_label_encoder=False,
    eval_metric="logloss",

    enable_categorical=True,
    tree_method="hist")

model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=True,
              eval_metric='logloss', gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=2,
              max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=50, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [107]:
explainer = dx.Explainer(model, X_test, y_test)

Preparation of a new explainer is initiated

  -> data              : 9769 rows 100 cols
  -> target variable   : Parameter 'y' was a pandas.DataFrame. Converted to a numpy.ndarray.
  -> target variable   : 9769 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7f80de993a30> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.000838, mean = 0.239, max = 0.997
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.953, mean = 0.000394, max = 0.997
  -> model_info        : package xgboost

A new explainer has been created!


In [108]:
explainer.model_performance()

,recall,precision,f1,accuracy,auc
XGBClassifier,0.585543,0.790872,0.672893,0.863753,0.917741


In [109]:
protected = X_test['gender_Male'].apply(lambda x: 'male' if x else 'female')
privileged = 'male'

fobject = explainer.model_fairness(protected=protected, privileged=privileged, label='XGBClassifier')

In [110]:
fobject.plot()

In [111]:
fobject.fairness_check()

Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
            TPR       ACC       PPV       FPR       STP
female  0.89916  1.130435  1.049618  0.197183  0.313043


In [112]:
model2 = RandomForestClassifier(n_estimators=100)
model2.fit(X_train, y_train)

/tmp/ipykernel_72913/3315918735.py:2: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestClassifier()

In [113]:
explainer2 = dx.Explainer(model2, X_test, y_test)

Preparation of a new explainer is initiated

  -> data              : 9769 rows 100 cols
  -> target variable   : Parameter 'y' was a pandas.DataFrame. Converted to a numpy.ndarray.
  -> target variable   : 9769 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7f80de993a30> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.


/home/x/Documents/eXplainableMachineLearning-2023/venv/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but RandomForestClassifier was fitted with feature names



  -> predicted values  : min = 0.0, mean = 0.241, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.99, mean = -0.0016, max = 1.0
  -> model_info        : package sklearn

A new explainer has been created!


In [114]:
explainer2.model_performance()

,recall,precision,f1,accuracy,auc
RandomForestClassifier,0.642002,0.735064,0.685388,0.858942,0.90453


In [115]:
fobject2 = explainer2.model_fairness(protected=protected, privileged=privileged, label='RandomForestClassifier')
fobject2.plot()

In [116]:
protected_train = X_train['gender_Male'].apply(lambda x: 'male' if x else 'female')

model3 = XGBClassifier(
    n_estimators=50,
    max_depth=2,
    use_label_encoder=False,
    eval_metric="logloss",

    enable_categorical=True,
    tree_method="hist")

resample_idxs = resample(
    protected=protected_train,
    y=y_train,
    type='uniform'
)

model3.fit(X_train.iloc[resample_idxs], y_train.iloc[resample_idxs])
explainer3 = dx.Explainer(
    model3,
    X_test,
    y_test,
)

explainer3.model_performance()

protected Series will be converted to np.ndarray
protected array is not string type, converting to string 
Parameter 'y' was a pandas.DataFrame. Converted to a numpy.ndarray.
Preparation of a new explainer is initiated

  -> data              : 9769 rows 100 cols
  -> target variable   : Parameter 'y' was a pandas.DataFrame. Converted to a numpy.ndarray.
  -> target variable   : 9769 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7f80de993a30> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.00151, mean = 0.234, max = 0.998
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.956, mean = 0.0055, max = 0.996
  -> model_info        : pa

,recall,precision,f1,accuracy,auc
XGBClassifier,0.570145,0.785967,0.660882,0.859965,0.911005


In [117]:
fobject3 = explainer3.model_fairness(
    protected=protected,
    privileged=privileged,
    label='XGBClassifier resample'
)

fobject3.plot()

In [118]:
fobject3.fairness_check()

Bias detected in 2 metrics: PPV, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC      PPV   FPR       STP
female  1.245902  1.114734  0.77657  0.94  0.584158


In [119]:
model3.fit(X_train, y_train, sample_weight=reweight(protected=protected_train, y=y_train))
explainer4 = dx.Explainer(
    model3,
    X_test,
    y_test,
)

explainer4.model_performance()

Parameter 'y' was a pandas.DataFrame. Converted to a numpy.ndarray.
protected Series will be converted to np.ndarray
protected array is not string type, converting to string 
Preparation of a new explainer is initiated

  -> data              : 9769 rows 100 cols
  -> target variable   : Parameter 'y' was a pandas.DataFrame. Converted to a numpy.ndarray.
  -> target variable   : 9769 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7f80de993a30> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.00136, mean = 0.233, max = 0.998
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.95, mean = 0.00603, max = 0.997
  -> model_info        : pa

,recall,precision,f1,accuracy,auc
XGBClassifier,0.56929,0.793206,0.662849,0.861398,0.911288


In [120]:
fobject4 = explainer4.model_fairness(
    protected=protected,
    privileged=privileged,
    label='XGBClassifier reweight'
)

fobject4.plot()

In [121]:
fobject4.fairness_check()

Bias detected in 3 metrics: TPR, PPV, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV       FPR       STP
female  1.303309  1.114458  0.768772  1.088889  0.619289


In [122]:
fobject.plot([fobject2, fobject3, fobject4])